In [1]:
import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.tree import DecisionTreeClassifier
import warnings            
warnings.filterwarnings("ignore")

In [2]:
def prepare_data():
    # verinin okunması
    data = pd.read_csv('M30_EURUSD.csv')
    data = data.replace("?", np.nan)
    # soru işaretleri olan satırlar veriden silindi. 
    # verinin feature larının ayrılması
    df = data.iloc[:,5:430]
    df_first = data.iloc[:,1:5]
    df_result = data.iloc[:,430:-1]
    return df, df_first, df_result

In [3]:
# parametrik fonk. tanımlaması
# 1 : 
# 2 :

def kategorikleri_dummy_yap(df):
    cat_column_names = ['ind_7','ind_11','ind_24','ind_38','ind_54','ind_57','ind_60','ind_63','ind_66','ind_69','ind_72','ind_75',
                    'ind_78','ind_81','ind_84','ind_87','ind_89','ind_91','ind_93','ind_95','ind_97','ind_99','ind_101',
                    'ind_103','ind_105','ind_107','ind_109', 'ind_111', 'ind_113', 'ind_115','ind_138','ind_141','ind_144',
                    'ind_157','ind_159','ind_161','ind_163','ind_165','ind_167','ind_169','ind_171','ind_173','ind_175',
                    'ind_177','ind_182','ind_184','ind_187','ind_190','ind_193','ind_196','ind_199','ind_202','ind_205',
                    'ind_208','ind_211','ind_213','ind_384','ind_386','ind_388','ind_390']
    # categorical kolonların dummy var. oalrak değiştirdik
    dms = pd.get_dummies(df[cat_column_names])
    dms_none_cols = dms.filter(regex = '_NONE').columns
    for i in dms_none_cols:
        dms.drop(i,axis=1,inplace=True)
    dms_red_cols = dms.filter(regex = '_RED').columns
    for i in dms_red_cols:
        dms.drop(i,axis=1,inplace=True)
    #datadan categorical olan kolonları çıkarıyoruz ve type nı değiştiriyoruz
    df_noncategoric = df.drop(cat_column_names,axis=1).astype("float64")
    from sklearn.preprocessing import Imputer 
    imputer= Imputer(missing_values='NaN', strategy = 'mean', axis=0 ) 
    imputer = imputer.fit(df_noncategoric)   
    df_noncategoric = imputer.transform(df_noncategoric) 
    df_noncategoric = pd.DataFrame(df_noncategoric)
    df_all = pd.concat([df_noncategoric, dms], axis=1)
    # y değerlerinin alınması
    return df_all, df_noncategoric, dms

In [4]:
df, df_first, df_result = prepare_data()
df_all, df_noncategoric, dms = kategorikleri_dummy_yap(df)
df_all.head()

,0,1,2,3,4,5,6,7,8,9,...,ind_211_BUY,ind_211_SELL,ind_384_BUY,ind_384_SELL,ind_386_BUY,ind_386_SELL,ind_388_BUY,ind_388_SELL,ind_390_BUY,ind_390_SELL
0,-0.143312,-0.143383,48.886194,48.096344,0.78985,49.815026,9.5422,9.542622,9.543487,9.544036,...,0,0,0,0,0,0,0,0,0,0
1,-0.143312,-0.143383,48.886194,48.096344,0.78985,49.815026,9.5422,9.542622,9.543487,9.544036,...,0,0,0,0,0,0,0,0,0,0
2,-0.143312,-0.143383,48.886194,48.096344,0.78985,49.815026,4.9500,9.542622,9.543487,9.544036,...,0,0,0,0,0,0,0,0,0,0
3,-0.143312,-0.143383,48.886194,48.096344,0.78985,49.815026,5.0500,9.542622,9.543487,9.544036,...,0,0,0,0,0,0,0,0,0,0
4,-0.143312,-0.143383,48.886194,48.096344,0.78985,49.815026,4.6000,9.542622,9.543487,9.544036,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_first.head()

,open,high,low,close
0,1.11894,1.11959,1.11894,1.11942
1,1.11943,1.11960,1.11913,1.11942
2,1.11940,1.11943,1.11891,1.11922
3,1.11924,1.11968,1.11919,1.11957
4,1.11955,1.11955,1.11914,1.11926


In [6]:

data = pd.read_csv('M30_EURUSD.csv')
df_diff = pd.DataFrame()

df_diff['x1'] = ( data['close'] - data['open'] ) / data['open'] 
df_diff['x2'] = ( data['high'] - data['low'] ) / data['low']
df_diff['x3'] =(  data['close'] - data['high'] ) /  data['high'] 
df_diff['x4'] = ( data['close'] - data['low'] ) / data['low'] 
df_diff['x5'] = ( data['open'] - data['high'] ) / data['high'] 
df_diff['x6'] = ( data['open'] - data['low'] ) / data['low'] 
df_diff = df_diff * 100
df_diff.head()

,x1,x2,x3,x4,x5,x6
0,0.042898,0.058091,-0.015184,0.042898,-0.058057,0.000000
1,-0.000893,0.041997,-0.016077,0.025913,-0.015184,0.026807
2,-0.016080,0.046474,-0.018760,0.027706,-0.002680,0.043793
3,0.029484,0.043782,-0.009824,0.033953,-0.039297,0.004468
4,-0.025903,0.036635,-0.025903,0.010723,0.000000,0.036635


In [7]:
df_newf = pd.DataFrame()
for i in df_diff.columns:
    for j in df_diff.columns:
        df_newf[i+'*'+j] = df_diff[i]*df_diff[j]

In [8]:
df = pd.concat([df_diff, df_newf], axis=1)
df.head()

,x1,x2,x3,x4,x5,x6,x1*x1,x1*x2,x1*x3,x1*x4,...,x5*x3,x5*x4,x5*x5,x5*x6,x6*x1,x6*x2,x6*x3,x6*x4,x6*x5,x6*x6
0,0.042898,0.058091,-0.015184,0.042898,-0.058057,0.000000,1.840216e-03,0.002492,-0.000651,0.001840,...,0.000882,-0.002491,0.003371,-0.000000,0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000
1,-0.000893,0.041997,-0.016077,0.025913,-0.015184,0.026807,7.980059e-07,-0.000038,0.000014,-0.000023,...,0.000244,-0.000393,0.000231,-0.000407,-0.000024,0.001126,-0.000431,0.000695,-0.000407,0.000719
2,-0.016080,0.046474,-0.018760,0.027706,-0.002680,0.043793,2.585678e-04,-0.000747,0.000302,-0.000446,...,0.000050,-0.000074,0.000007,-0.000117,-0.000704,0.002035,-0.000822,0.001213,-0.000117,0.001918
3,0.029484,0.043782,-0.009824,0.033953,-0.039297,0.004468,8.693235e-04,0.001291,-0.000290,0.001001,...,0.000386,-0.001334,0.001544,-0.000176,0.000132,0.000196,-0.000044,0.000152,-0.000176,0.000020
4,-0.025903,0.036635,-0.025903,0.010723,0.000000,0.036635,6.709791e-04,-0.000949,0.000671,-0.000278,...,-0.000000,0.000000,0.000000,0.000000,-0.000949,0.001342,-0.000949,0.000393,0.000000,0.001342


In [9]:
y = data['220_signal'].shift(periods=1)
df = df.iloc[1:-1,:]
y = y.iloc[1:-1]
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size = 0.20, shuffle = False)

In [10]:
y.head()

1    BUY
2    BUY
3    BUY
4    BUY
5    BUY
Name: 220_signal, dtype: object

In [11]:
logreg = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
log = logreg.fit(X_train, y_train)
y_pred = log.predict(X_test)
confusion_mat = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: ",accuracy)
print('-------------------------------')
print("Counfusion matrix: \n",confusion_mat)
print('-------------------------------')
print('Classification report')
print(classification_report(y_test, y_pred))

Accuracy:  0.41811037685300945
-------------------------------
Counfusion matrix: 
 [[ 506    0  826]
 [ 549    0 1149]
 [ 734    0 1835]]
-------------------------------
Classification report
              precision    recall  f1-score   support

         BUY       0.28      0.38      0.32      1332
        NONE       0.00      0.00      0.00      1698
        SELL       0.48      0.71      0.58      2569

    accuracy                           0.42      5599
   macro avg       0.25      0.36      0.30      5599
weighted avg       0.29      0.42      0.34      5599



In [12]:
df, df_first, df_result = prepare_data()
df.head()

,ind_5,ind_6,ind_7,ind_8,ind_9,ind_10,ind_11,ind_12,ind_13,ind_14,...,ind_420,ind_421,ind_422,ind_423,ind_424,ind_425,ind_426,ind_427,ind_428,ind_429
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.949999999999122,...,NaN,NaN,NaN,NaN,0.0434782608689356,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.049999999999777,...,NaN,NaN,NaN,NaN,0.5319148936172323,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6000000000001595,...,NaN,NaN,NaN,NaN,0.8333333333337034,0.0,NaN,NaN,NaN,NaN
